# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [1]:
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# Filter the DataFrame
filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]

# Display the filtered DataFrame
filtered_df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [2]:
# Filter the DataFrame for customers who responded "Yes"
responded_yes_df = df[df['Response'] == 'Yes']

# Group by 'Policy Type' and 'Gender' and calculate the average 'Total Claim Amount'
average_claims = responded_yes_df.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean().reset_index()

# Display the result
average_claims

,Policy Type,Gender,Total Claim Amount
0,Corporate Auto,F,433.738499
1,Corporate Auto,M,408.582459
2,Personal Auto,F,452.965929
3,Personal Auto,M,457.010178
4,Special Auto,F,453.280164
5,Special Auto,M,429.527942


Based on the provided data, here are some conclusions:

1. **Corporate Auto**:
   - Female customers have a slightly higher average total claim amount ($433.74) compared to male customers ($408.58).

2. **Personal Auto**:
   - Male customers have a slightly higher average total claim amount ($457.01) compared to female customers ($452.97).

3. **Special Auto**:
   - Female customers have a slightly higher average total claim amount ($453.28) compared to male customers ($429.53).

### General Observations:
- For **Corporate Auto** policies, females tend to have higher average claim amounts than males.
- For **Personal Auto** policies, males tend to have higher average claim amounts than females.
- For **Special Auto** policies, females tend to have higher average claim amounts than males.

### Marketing Implications:
- The insurance company might consider tailoring their marketing strategies based on these insights. For example, they could emphasize the benefits of Corporate Auto policies to female customers and Personal Auto policies to male customers.
- Understanding these differences can help in designing more effective marketing campaigns and customer service strategies to address the specific needs and behaviors of different customer segments.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [3]:
# Group by 'State' and count the number of customers
state_counts = df.groupby('State').size().reset_index(name='Customer Count')

# Filter the results to include only states with more than 500 customers
filtered_states = state_counts[state_counts['Customer Count'] > 500]

# Display the result
filtered_states

,State,Customer Count
0,Arizona,1937
1,California,3552
2,Nevada,993
3,Oregon,2909
4,Washington,888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [4]:
# Group by 'Education' and 'Gender' and calculate the max, min, and median 'Customer Lifetime Value'
clv_stats = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).reset_index()

# Display the result
clv_stats

,Education,Gender,max,min,median
0,Bachelor,F,73225.95652,1904.000852,5640.505303
1,Bachelor,M,67907.27050,1898.007675,5548.031892
2,College,F,61850.18803,1898.683686,5623.611187
3,College,M,61134.68307,1918.119700,6005.847375
4,Doctor,F,44856.11397,2395.570000,5332.462694
5,Doctor,M,32677.34284,2267.604038,5577.669457
6,High School or Below,F,55277.44589,2144.921535,6039.553187
7,High School or Below,M,83325.38119,1940.981221,6286.731006
8,Master,F,51016.06704,2417.777032,5729.855012
9,Master,M,50568.25912,2272.307310,5579.099207


Based on the provided data, here are some conclusions regarding the customer lifetime value (CLV) by education level and gender:

### General Observations:
1. **Bachelor's Degree**:
   - Female customers have a higher maximum CLV ($73,225.96) compared to male customers ($67,907.27).
   - The median CLV is slightly higher for female customers ($5,640.51) compared to male customers ($5,548.03).

2. **College**:
   - Female customers have a higher maximum CLV ($61,850.19) compared to male customers ($61,134.68).
   - The median CLV is higher for male customers ($6,005.85) compared to female customers ($5,623.61).

3. **Doctorate**:
   - Female customers have a higher maximum CLV ($44,856.11) compared to male customers ($32,677.34).
   - The median CLV is slightly higher for male customers ($5,577.67) compared to female customers ($5,332.46).

4. **High School or Below**:
   - Male customers have a significantly higher maximum CLV ($83,325.38) compared to female customers ($55,277.45).
   - The median CLV is higher for male customers ($6,286.73) compared to female customers ($6,039.55).

5. **Master's Degree**:
   - Female customers have a slightly higher maximum CLV ($51,016.07) compared to male customers ($50,568.26).
   - The median CLV is slightly higher for female customers ($5,729.86) compared to male customers ($5,579.10).

### Conclusions:
- **High School or Below**: Male customers in this education category have the highest maximum CLV ($83,325.38) among all groups, indicating that this segment might include some highly valuable customers.
- **Bachelor's Degree**: Female customers with a Bachelor's degree have the highest maximum CLV ($73,225.96) among female customers, suggesting that this segment is particularly valuable.
- **Doctorate**: Female customers with a Doctorate degree have a higher maximum CLV compared to their male counterparts, but the median CLV is higher for males.
- **College and Master's Degree**: The differences in maximum and median CLV between genders are relatively small, indicating a more balanced distribution of customer value in these education levels.

### Marketing Implications:
- The insurance company might consider focusing marketing efforts on male customers with a high school education or below, as they have the highest maximum CLV.
- Female customers with a Bachelor's degree also represent a valuable segment and could be targeted with tailored marketing campaigns.
- Understanding these differences can help in designing more effective marketing strategies and customer retention programs to maximize the lifetime value of different customer segments.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [6]:
# Ensure the date column is in datetime format with specified format
df['Effective To Date'] = pd.to_datetime(df['Effective To Date'], format='%m/%d/%Y')

# Extract the month from the date column
df['Month'] = df['Effective To Date'].dt.month

# Group by 'State' and 'Month' and count the number of policies sold
policies_by_state_month = df.groupby(['State', 'Month']).size().reset_index(name='Policy Count')

# Pivot the table to get months as columns and states as rows
pivot_table = policies_by_state_month.pivot(index='State', columns='Month', values='Policy Count').fillna(0)

# Display the result
pivot_table

Month,1,2
State,,
Arizona,1008,929
California,1918,1634
Nevada,551,442
Oregon,1565,1344
Washington,463,425


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [7]:
# Extract the month from the date column
df['Month'] = df['Effective To Date'].dt.month

# Group by 'State' and 'Month' and count the number of policies sold
policies_by_state_month = df.groupby(['State', 'Month']).size().reset_index(name='Policy Count')

# Sum the total number of policies sold by state
total_policies_by_state = policies_by_state_month.groupby('State')['Policy Count'].sum().reset_index()

# Sort the states by the total number of policies sold in descending order
top_states = total_policies_by_state.sort_values(by='Policy Count', ascending=False).head(3)

# Filter the original grouped data to include only the top 3 states
top_states_policies = policies_by_state_month[policies_by_state_month['State'].isin(top_states['State'])]

# Pivot the table to get months as columns and states as rows
pivot_table_top_states = top_states_policies.pivot(index='State', columns='Month', values='Policy Count').fillna(0)

# Display the result
pivot_table_top_states

Month,1,2
State,,
Arizona,1008,929
California,1918,1634
Oregon,1565,1344


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [8]:
df['Sales Channel'].unique()

array(['Agent', 'Call Center', 'Branch', 'Web'], dtype=object)

In [15]:
df.dtypes

Unnamed: 0                                int64
Customer                                 object
State                                    object
Customer Lifetime Value                 float64
Response                                 object
Coverage                                 object
Education                                object
Effective To Date                datetime64[ns]
EmploymentStatus                         object
Gender                                   object
Income                                    int64
Location Code                            object
Marital Status                           object
Monthly Premium Auto                      int64
Months Since Last Claim                 float64
Months Since Policy Inception             int64
Number of Open Complaints               float64
Number of Policies                        int64
Policy Type                              object
Policy                                   object
Renew Offer Type                        

In [14]:
# Ensure the 'Response' column is in the correct format
df['Response'] = df['Response'].astype(str)

# Identify the marketing channel column
marketing_channel_column = 'Sales Channel'

# Filter the data to include only customers who responded "Yes"
responded_yes_df = df[df['Response'] == 'Yes']

# Calculate the response count for each marketing channel
response_count = responded_yes_df.groupby(marketing_channel_column).size().reset_index(name='Response Count')

# Calculate the total number of customers for each marketing channel
total_customers = df.groupby(marketing_channel_column).size().reset_index(name='Total Count')

# Merge the response count and total count dataframes
merged_df = pd.merge(response_count, total_customers, on=marketing_channel_column)

# Calculate the response rate
merged_df['Response Rate'] = merged_df['Response Count'] / merged_df['Total Count']

# Display the result
merged_df

,Sales Channel,Response Count,Total Count,Response Rate
0,Agent,742,4121,0.180053
1,Branch,326,3022,0.107876
2,Call Center,221,2141,0.103223
3,Web,177,1626,0.108856


Here are some conclusions regarding the effectiveness of different marketing channels on the customer response rate:

### Data Summary:
- **Agent**:
  - Response Count: 742
  - Total Count: 4121
  - Response Rate: 18.01%

- **Branch**:
  - Response Count: 326
  - Total Count: 3022
  - Response Rate: 10.79%

- **Call Center**:
  - Response Count: 221
  - Total Count: 2141
  - Response Rate: 10.32%

- **Web**:
  - Response Count: 177
  - Total Count: 1626
  - Response Rate: 10.89%

### Conclusions:
1. **Agent Channel**:
   - The Agent channel has the highest response rate at 18.01%.
   - This indicates that customers are more likely to respond positively when contacted through an agent compared to other channels.
   - The high response rate suggests that personal interaction through agents is effective in engaging customers.

2. **Branch Channel**:
   - The Branch channel has a response rate of 10.79%.
   - While lower than the Agent channel, it is still relatively effective.
   - This suggests that in-person interactions at branches can also be a good way to engage customers, though not as effective as agents.

3. **Call Center Channel**:
   - The Call Center channel has a response rate of 10.32%.
   - This is slightly lower than the Branch channel.
   - It indicates that while call centers are useful, they might not be as effective as personal or in-person interactions.

4. **Web Channel**:
   - The Web channel has a response rate of 10.89%.
   - This is comparable to the Branch and Call Center channels.
   - It suggests that online interactions are as effective as in-person and call center interactions, but still less effective than agent interactions.

### Marketing Implications:
- **Focus on Agent Channel**: Given the highest response rate, the marketing team should consider investing more in the Agent channel. Training and expanding the agent network could yield higher customer engagement and response rates.
- **Enhance Branch and Web Channels**: Since the Branch and Web channels have similar response rates, efforts to improve customer experience in these channels could help increase their effectiveness. This could include better online tools, more personalized in-branch services, and targeted marketing campaigns.
- **Optimize Call Center Operations**: While the Call Center channel has the lowest response rate, it is still a significant channel. Improving call scripts, training call center staff, and using data analytics to target calls more effectively could help improve response rates.

Overall, the data suggests that personal interactions through agents are the most effective way to engage customers, followed by in-person branch interactions and online/web interactions.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9